[this doc on github](https://github.com/dotnet/interactive/tree/master/samples/notebooks/powershell)

# Charts with XPlot using hashtable casting <img src="https://raw.githubusercontent.com/PowerShell/PowerShell/master/assets/Powershell_black_64.png" align="right"/>
Charts can be rendered using [Xplot.Plotly](https://fslab.org/XPlot/). 
We will cover some example on how to use XPlot in a notebook with .NET Interactive.

> NOTE: This and "Plotting with Xplot using constructors" produce the same output. They're just using different scripting mechanisms.

In [ ]:
#!csharp
#r "nuget: XPlot.Plotly.Interactive, 4.0.2"

# Rendering Scatter plots
One of the most commonly used type of chart to explore data set. Use the type `Scatter`.

In [ ]:
$openSeries = [Graph.Scatter]@{
    name = "Open"
    x = @(1, 2, 3, 4)
    y = @(10, 15, 13, 17)
}

$closeSeries = [Graph.Scatter]@{
    name = "Close"
    x = @(2, 3, 4, 5)
    y = @(16, 5, 11, 9)
}

$chart = @($openSeries, $closeSeries) | New-PlotlyChart -Title "Open vs Close"
Out-Display $chart

Let's change it to be markers style, so more like a scatter plot.

In [ ]:
$openSeries.mode = "markers";
$closeSeries.mode = "markers";

$chart = @($openSeries, $closeSeries) | New-PlotlyChart -Title "Open vs Close"
Out-Display $chart

`Scatter` can also produce polar charts by setting the radial property `r` and angular proeprty `t`

In [ ]:
$openSeries = [Graph.Scatter]@{
    name = "Open"
    r = @(1, 2, 3, 4)
    t = @(45, 100, 150, 290)
}

$closeSeries = [Graph.Scatter]@{
    name = "Close"
    r = @(2, 3, 4, 5)
    t = @(16, 45, 118, 90)
}

$layout = [Layout]@{
    title = "Open vs Close"
    orientation = -90
}

$chart = @($openSeries, $closeSeries) | New-PlotlyChart -Layout $layout
$chart | Out-Display

## Large scatter plots and performance
It is not uncommon to have scatter plots with a large dataset, it is a common scenario at the beginning of a data exploration process. Using the default `svg` based rendering will create performace issues as the dom will become very large.
We can then use `web-gl` support to address the problem.

In [ ]:
#!time
$series = 1..10 | ForEach-Object {
    [Graph.Scattergl]@{
        name = "Series $_"
        mode = "markers"
        x = [double[]](Get-Random -Count 100000 -Minimum -100000 -Maximum 100000)
        y = [double[]](Get-Random -Count 100000 -Minimum -100000 -Maximum 100000)
    }
}

New-PlotlyChart -Title "Large Dataset" -Trace $series | Out-Display

Can provide custom marker `colour`, `size` and `colorscale` to display even more information to the user.

In [ ]:
$series | ForEach-Object {
    [int[]] $sizes = Get-Random -Count 100 -Minimum 0.0 -Maximum 1.0 |
        ForEach-Object { $_ -lt 0.75 ? (Get-Random -Minimum 1 -Maximum 5) : (Get-Random -Minimum 10 -Maximum 15) }
    $temperatures = $sizes | ForEach-Object { ($_ * 10) - 100 }

    $_.x = [double[]](Get-Random -Count 100000 -Minimum -100000 -Maximum 100000)
    $_.y = [double[]](Get-Random -Count 100000 -Minimum -100000 -Maximum 100000)
    
    $_.marker = [XPlot.Plotly.Marker]@{
        size = $sizes
        color = $temperatures
        colorscale = "hot"
    }
}

New-PlotlyChart -Title "Large Dataset" -Trace $series | Out-Display

Plotly pvoides some additional `color scales` to use.

In [ ]:
foreach ($trace in $series) {
    $trace.marker.colorscale = "Viridis"
}

New-PlotlyChart -Title "Viridis scale" -Trace $series | Out-Display

foreach ($trace in $series) {
    $trace.marker.colorscale = "Hot"
}

New-PlotlyChart -Title "Hot scale" -Trace $series | Out-Display

foreach ($trace in $series) {
    $trace.marker.colorscale = "Jet"
}

New-PlotlyChart -Title "Jet scale" -Trace $series | Out-Display

# Rendering Histograms
Let's have a look at using histograms, the next cell sets up some generators.

In [ ]:
$count = 20
[datetime[]] $dates = 1..$count | ForEach-Object { (Get-Date).AddMinutes((Get-Random -Minimum $_ -Maximum ($_+30))) }

Now let's define histogram traces:

In [ ]:
$openByTime = [Graph.Histogram]@{
    name = "Open"
    x = $dates
    y = [double[]](Get-Random -Count $count -Minimum 0 -Maximum 200)
}

$closeByTime = [Graph.Histogram]@{
    name = "Close"
    x = $dates
    y = [double[]](Get-Random -Count $count -Minimum 0 -Maximum 200)
}

New-PlotlyChart -Trace @($openByTime, $closeByTime) | Out-Display

The Histogram generator will automatically count the number of items per bin. 

Setting `histfunc` to `"sum"` we can now add up all the values contained in each bin.
Note that we are creatng bin using the `x` data point and we are using bydefault autobinx

In [ ]:
$openByTime.histfunc = 'sum'
$closeByTime.histfunc = 'sum'

(New-PlotlyChart -Trace @($openByTime, $closeByTime)) | Out-Display

# Area chart and Polar Area chart

By populating hte property `fill` of a `Scatter` trace the chart will render as area chart.

Here is set to `"tozeroy"` which will create a fill zone underneath the line reachin to the 0 of the y axis.

In [ ]:
$openSeries = [Graph.Scatter]@{
    name = "Open"
    x = @(1, 2, 3, 4)
    y = @(10, 15, 13, 17)
    fill = "tozeroy"
    mode = "lines"
}

$closeSeries = [Graph.Scatter]@{
    name = "Close"
    x = @(1, 2, 3, 4)
    y = @(3, 5, 11, 9)
    fill = "tozeroy"
    mode = "lines"
}

$chart = @($openSeries, $closeSeries) | New-PlotlyChart -Title "Open vs Close"
Out-Display $chart

With one `fill` set to `"tonexty"` the cahrt will fill the aread between traces.

In [ ]:
$openSeries.fill = $null;
$closeSeries.fill = "tonexty";

$chart = @($openSeries, $closeSeries) | New-PlotlyChart -Title "Open vs Close"
Out-Display $chart

Using `Area` traces we can generate radial area chart. In this example we are using cardinal points to xpress angular values.
The array `{"North", "N-E", "East", "S-E", "South", "S-W", "West", "N-W"}` will be autoimatically translated to angular values.

In [ ]:
$areaTrace1 = [Graph.Area]@{
    r = @(77.5, 72.5, 70.0, 45.0, 22.5, 42.5, 40.0, 62.5)
    t = @("North", "N-E", "East", "S-E", "South", "S-W", "West", "N-W")
    name = "11-14 m/s"
    marker = [XPlot.Plotly.Marker]@{
        color = "rgb(106,81,163)"
    }
}

$areaTrace2 = [Graph.Area]@{
    r = @(57.49999999999999, 50.0, 45.0, 35.0, 20.0, 22.5, 37.5, 55.00000000000001)
    t = @("North", "N-E", "East", "S-E", "South", "S-W", "West", "N-W")
    name = "8-11 m/s"
    marker = [XPlot.Plotly.Marker]@{
        color = "rgb(158,154,200)"
    }
}

$areaTrace3 = [Graph.Area]@{
    r = @(40.0, 30.0, 30.0, 35.0, 7.5, 7.5, 32.5, 40.0)
    t = @("North", "N-E", "East", "S-E", "South", "S-W", "West", "N-W")
    name = "5-8 m/s"
    marker = [XPlot.Plotly.Marker]@{
        color = "rgb(203,201,226)"
    }
}

$areaTrace4 = [Graph.Area]@{
    r = @(20.0, 7.5, 15.0, 22.5, 2.5, 2.5, 12.5, 22.5)
    t = @("North", "N-E", "East", "S-E", "South", "S-W", "West", "N-W")
    name = "< 5 m/s"
    marker = [XPlot.Plotly.Marker]@{
        color = "rgb(242,240,247)"
    }
}

$areaLayout = [Layout]@{
    title = "Wind Speed Distribution in Laurel, NE"
    font = [XPlot.Plotly.Font]@{
        size = 16
    }
    legend = [XPlot.Plotly.Legend]@{
        font = [XPlot.Plotly.Font]@{
            size = 16
        }
    }
    radialaxis = [XPlot.Plotly.Radialaxis]@{
        ticksuffix = "%"
    }
    orientation = 270
}

New-PlotlyChart -Layout $areaLayout -Trace @($areaTrace1, $areaTrace2, $areaTrace3, $areaTrace4) | Out-Display